In [164]:
import pandas as pd
from core.repository import load_financial
from base.timeutil import YearMonth
import sqlite3
from core.repository.maria.conn import maria_home

db_file = "analysis.db"

try:
    layer1 = pd.read_sql("select * from layer1", sqlite3.connect(db_file))
except:
    print("Fetching month chart...")
    month_chart = pd.read_sql("""
        select month_chart.*, stock.name, stock.exchange from month_chart
        join stock on month_chart.code = stock.code;
    """, maria_home())
    print("Building layer1...")

    begin = YearMonth(2002, 4)
    end = YearMonth(2023, 3)

    month_chart["년월"] = [YearMonth.from_date(d) for d in month_chart["date"]]
    layer1 = pd.DataFrame()
    for ym1, ym2 in [(row.iloc[0], row.iloc[1]) for row in pd.Series(begin.to(end)).rolling(2) if len(row) == 2]:
        print(f"\r{ym1} {ym2}", end="\r")
        df1 = month_chart[month_chart["년월"] == ym1].set_index("code")
        df2 = month_chart[month_chart["년월"] == ym2].set_index("code")
        df1 = df1[df1["val_last"] >= 1000_0000]
        df2.loc[df2["val_last"] == 0, "close"] = 0

        df = pd.DataFrame({
            "종목명": df1["name"],
            "거래소": df1["exchange"],
            "매수년월": str(ym1),
            "매도년월": str(ym2),
            "시가총액": df1["cap"],
            "매수일": df1["date"],
            "매수가": df1["close"],
            "거래량": df1["vol"],
            "거래대금": df1["val"],
        })

        df["매도가"] = df2["close"]
        df["매도일"] = df2["date"]
        df["수익률"] = df["매도가"] / df["매수가"] - 1

        # 재무데이터 조인하여 layer1에 반영
        fn = load_financial(ym1.year, ym1.month)
        layer1 = pd.concat([layer1, df.join(fn)])

    layer1 = layer1.reset_index()
    layer1["확정실적"] = layer1["확정실적"].apply(str)
    layer1.to_sql("layer1", sqlite3.connect(db_file), if_exists="replace", index=False)

layer1

,code,종목명,거래소,매수년월,매도년월,시가총액,매수일,매수가,거래량,거래대금,...,E_QoQ,R/A_QoQ,GP/A_QoQ,O/A_QoQ,E/A_QoQ,R/EQ_QoQ,GP/EQ_QoQ,O/EQ_QoQ,E/EQ_QoQ,확정실적
0,060310,3S,코스닥,2002-04,2002-05,14328000000,2002-04-30,2604,1480725,12071273140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1,006840,AK홀딩스,유가증권,2002-04,2002-05,108300000000,2002-04-30,11302,706504,15311799550,...,2.151386,0.039735,0.013609,0.007998,0.012766,0.081359,0.027205,0.015848,0.025052,2001-4Q
2,054620,APS홀딩스,코스닥,2002-04,2002-05,20934600000,2002-04-30,6162,3623832,41050604910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
3,001460,BYC,유가증권,2002-04,2002-05,38413822500,2002-04-30,61500,73712,5506711000,...,-0.608231,-0.011676,-0.001393,0.003334,-0.017559,-0.014840,-0.001662,0.004425,-0.023219,2001-4Q
4,001040,CJ,유가증권,2002-04,2002-05,1250135164800,2002-04-30,28006,4513180,263372467300,...,1.157134,0.248107,0.137269,0.058049,0.032612,0.635466,0.395791,0.181644,0.106711,2001-4Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387473,000540,흥국화재,유가증권,2023-02,2023-03,239946279075,2023-02-28,3735,1800328,6600268565,...,NaN,0.005376,NaN,0.001088,NaN,0.380950,NaN,0.033848,NaN,2022-3Q
387474,000547,흥국화재2우B,유가증권,2023-02,2023-03,3070464000,2023-02-28,19990,105811,2268502740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
387475,003280,흥아해운,유가증권,2023-02,2023-03,352222477035,2023-02-28,1465,7927404,12011059811,...,8.103269,0.085515,0.043808,0.042733,0.018525,0.229167,0.108547,0.102940,0.046502,2022-3Q
387476,037440,희림,코스닥,2023-02,2023-03,138807075750,2023-02-28,9970,17197648,176303839820,...,2.517199,-0.004923,0.011450,0.005833,0.010255,-0.045885,0.030324,0.016097,0.028984,2022-3Q


In [165]:

import numpy as np

layer2 = layer1.rename(
    columns={
        "시가총액": "P",
        "거래대금": "TV",
        "자산총계": "A",
        "자본총계": "EQ",

        "매출액": "R",
        "매출총이익": "GP",
        "영업이익": "O",
        "당기순이익": "E",
    }
)

factors = [
    "R_QoQ",
    "GP_QoQ",
    "O_QoQ",
    "E_QoQ",

    "R/A_QoQ",
    "GP/A_QoQ",
    "O/A_QoQ",
    "E/A_QoQ",

    "R/EQ_QoQ",
    "GP/EQ_QoQ",
    "O/EQ_QoQ",
    "E/EQ_QoQ",

    "BIS",
    "BIS_QoQ",
]

for pos in ["R", "GP", "O", "EQ"]:
    factor = f"{pos}/P"
    factors.append(factor)
    layer2[factor] = layer2[pos] / layer2["P"]
    layer2.loc[layer2[pos] <= 0, factor] = np.nan

for neg in ["A", "EQ"]:
    for pos in ["R", "GP", "O", "E"]:
        factor = f"{pos}/{neg}"
        factors.append(factor)
        layer2[factor] = layer2[pos] / layer2[neg]
        layer2.loc[layer2[neg] <= 0, factor] = np.nan

factors.append("P")

layer2["TV/P"] = layer2["TV"] / layer2["P"]
factors.append("TV/P")

result = pd.DataFrame()
yms = layer2["매수년월"].unique()

from core.strategy import recipe

recipes = [
    {
        "P": -4,
        "GP/P": 4,
        "EQ/P": 2,
        "GP_QoQ": 1,
        "O_QoQ": 1,
        "GP/A_QoQ": 1,
        "O/A_QoQ": 1,
    },
    {
        "P": -4,
        "R/P": 2,
        "GP/P": 2,
        "EQ/P": 2,

        "R_QoQ": 1,
        "GP_QoQ": 1,
        "O_QoQ": 1,
        "E_QoQ": 1,

        "R/A_QoQ": 1,
        "GP/A_QoQ": 1,
        "O/A_QoQ": 1,
        "E/A_QoQ": 1
    },
    {
        "P": -8,
        "R/P": 4,
        "GP/P": 4,
        "EQ/P": 4,

        "R_QoQ": 1,
        "GP_QoQ": 1,
        "O_QoQ": 1,
        "E_QoQ": 1,

        "R/A_QoQ": 1,
        "GP/A_QoQ": 1,
        "O/A_QoQ": 1,
        "E/A_QoQ": 1
    },
    {
        "P": -8,
        "GP/P": 4,
        "EQ/P": 4,

        "R_QoQ": 1,
        "GP_QoQ": 1,
        "O_QoQ": 1,
        "E_QoQ": 1,

        "R/A_QoQ": 1,
        "GP/A_QoQ": 1,
        "O/A_QoQ": 1,
        "E/A_QoQ": 1
    },
]

print(pd.DataFrame(recipes))

for ym in yms:
    df = layer2[layer2["매수년월"] == ym].copy()
    for factor in factors:
        print(f"\r{ym}", end="")
        colname_pct = f"{factor}_pct"
        df[colname_pct] = np.ceil(df[factor].rank(method="min", pct=True) * 100)

    for i in range(len(recipes)):
        factor = f"super{i}"
        recipe = recipes[i]
        # 1. 레시피를 구성하는 개별 팩터 분위(percentile) * 가중치의 총합을 구함
        sv = sum([df[f"{k}_pct"] * w if w > 0 else (1 - df[f"{k}_pct"]) * abs(w) for k, w in recipe.items()])
        # 2. 위의 시리즈에 가중치의 총합을 나눈다 => 0~1 사이 값으로 일반화됨
        df[factor] = sv / sum([abs(w) for w in recipe.values()])
        df[f"{factor}_pct"] = np.ceil(df[factor].rank(method="min", pct=True) * 100)

    result = pd.concat([result, df])

factors = [f"super{i}" for i in range(len(recipes))] + factors
result

   P  GP/P  EQ/P  GP_QoQ  O_QoQ  GP/A_QoQ  O/A_QoQ  R/P  R_QoQ  E_QoQ  \
0 -4     4     2       1      1         1        1  NaN    NaN    NaN   
1 -4     2     2       1      1         1        1  2.0    1.0    1.0   
2 -8     4     4       1      1         1        1  4.0    1.0    1.0   
3 -8     4     4       1      1         1        1  NaN    1.0    1.0   

   R/A_QoQ  E/A_QoQ  
0      NaN      NaN  
1      1.0      1.0  
2      1.0      1.0  
3      1.0      1.0  
2023-02

,code,종목명,거래소,매수년월,매도년월,P,매수일,매수가,거래량,TV,...,P_pct,TV/P_pct,super0,super0_pct,super1,super1_pct,super2,super2_pct,super3,super3_pct
0,060310,3S,코스닥,2002-04,2002-05,14328000000,2002-04-30,2604,1480725,12071273140,...,19.0,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,006840,AK홀딩스,유가증권,2002-04,2002-05,108300000000,2002-04-30,11302,706504,15311799550,...,79.0,18.0,20.428571,45.0,33.444444,60.0,22.285714,45.0,16.333333,47.0
2,054620,APS홀딩스,코스닥,2002-04,2002-05,20934600000,2002-04-30,6162,3623832,41050604910,...,34.0,94.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001460,BYC,유가증권,2002-04,2002-05,38413822500,2002-04-30,61500,73712,5506711000,...,53.0,18.0,43.285714,87.0,40.000000,74.0,37.214286,80.0,30.250000,76.0
4,001040,CJ,유가증권,2002-04,2002-05,1250135164800,2002-04-30,28006,4513180,263372467300,...,97.0,29.0,30.928571,67.0,41.666667,77.0,27.357143,59.0,20.750000,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387473,000540,흥국화재,유가증권,2023-02,2023-03,239946279075,2023-02-28,3735,1800328,6600268565,...,68.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387474,000547,흥국화재2우B,유가증권,2023-02,2023-03,3070464000,2023-02-28,19990,105811,2268502740,...,1.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387475,003280,흥아해운,유가증권,2023-02,2023-03,352222477035,2023-02-28,1465,7927404,12011059811,...,78.0,16.0,14.500000,32.0,32.333333,60.0,14.285714,32.0,12.666667,39.0
387476,037440,희림,코스닥,2023-02,2023-03,138807075750,2023-02-28,9970,17197648,176303839820,...,51.0,92.0,23.500000,53.0,34.055556,65.0,23.964286,54.0,17.458333,51.0


In [197]:
import matplotlib.pyplot as plt
from core.base.quantutil import cagr, mdd
from base.timeutil import YearMonth
from datetime import date

plt.ioff()

# plt.rc("font", family="Malgun Gothic")
plt.rc('axes', unicode_minus=False)
plt.rcParams['grid.color'] = (0.5, 0.5, 0.5, 0.1)
plt.rcParams['agg.path.chunksize'] = 10_000

target_factors = factors
# target_factors = [f for f in factors if f.startswith("super")]
fig, subplots = plt.subplots(len(target_factors), 2)
fig.set_figwidth(16), fig.set_figheight(6 * len(target_factors))

summary = pd.DataFrame()
for i in range(len(target_factors)):
    print(f"\r{i + 1}/{len(target_factors)}", end="")
    factor = factors[i]
    colname_pct = f"{factor}_pct"
    percentiles = sorted(set(result[colname_pct].dropna()))
    mean_by_pct = []
    cagr_by_pct = []
    mdd_by_pct = []
    for pct in percentiles:
        by_pct = result[result[colname_pct] == pct]
        return_by_ym = by_pct.groupby("매도년월")["수익률"].mean()
        begin = YearMonth.from_string(by_pct["매수년월"].min())
        cumprod = (return_by_ym + 1).cumprod()
        last = cumprod[-1]
        mean_by_pct.append(return_by_ym.mean())
        cagr_by_pct.append(cagr(1, last, by_pct["매도년월"].unique().size / 12))
        dates = [date(int(s.split("-")[0]), int(s.split("-")[1]), 1) for s in return_by_ym.index]
        mdd_by_pct.append(mdd(dates, cumprod.values)[0])

    ax1, ax2 = subplots[i][0], subplots[i][1]

    cagr_by_pct_series = pd.Series(cagr_by_pct)
    mdd_by_pct_series = pd.Series(mdd_by_pct)

    ax1.set_xlabel(colname_pct)
    ax1.set_ylabel("CAGR")
    ax1.grid(True)
    ax1.bar(percentiles, cagr_by_pct, color=["tab:blue" if v > 0 else "tab:red" for v in cagr_by_pct])
    ax1.plot(percentiles, [cagr_by_pct_series.mean() for _ in percentiles], alpha=0.5)
    ax1.plot(percentiles, cagr_by_pct_series.rolling(5, center=True).mean())

    ax2.set_xlabel(colname_pct)
    ax2.set_ylabel("MDD")
    ax2.grid(True)
    ax2.bar(percentiles, mdd_by_pct, color=["tab:blue" if v > 0 else "tab:red" for v in mdd_by_pct])
    ax2.plot(percentiles, [mdd_by_pct_series.mean() for _ in percentiles], alpha=0.5)

    corr = pd.DataFrame({colname_pct: percentiles, "cagr": cagr_by_pct}).corr(method="spearman")
    cagr_by_pct_desc = cagr_by_pct_series[cagr_by_pct_series.index.sort_values(ascending=False)]
    mdd_by_pct_desc = mdd_by_pct_series[mdd_by_pct_series.index.sort_values(ascending=False)]
    summary = pd.concat([
        summary,
        pd.Series({
            "spearman": corr[colname_pct]["cagr"],

            "h3": cagr_by_pct_desc.head(3).mean(),
            "h5": cagr_by_pct_desc.head(5).mean(),
            "h10": cagr_by_pct_desc.head(10).mean(),

            "t3": cagr_by_pct_desc.tail(3).mean(),
            "t5": cagr_by_pct_desc.tail(5).mean(),
            "t10": cagr_by_pct_desc.tail(10).mean(),

            "mdd5": mdd_by_pct_desc.head(5).mean(),

            "h3-t3": cagr_by_pct_desc.head(3).mean() - cagr_by_pct_desc.tail(3).mean(),
            "h10-t10": cagr_by_pct_desc.head(10).mean() - cagr_by_pct_desc.tail(10).mean(),

            "h3-t3+mdd3": cagr_by_pct_desc.head(3).mean() - cagr_by_pct_desc.tail(3).mean() + mdd_by_pct_desc.head(
                3).mean(),
            "h10-t10+mdd10": cagr_by_pct_desc.head(10).mean() - cagr_by_pct_desc.tail(10).mean() + mdd_by_pct_desc.head(
                10).mean(),
            "mean": cagr_by_pct_desc.mean(),
            "median": cagr_by_pct_desc.median(),
        }).to_frame(factor).T
    ])

fig.savefig('test.png', bbox_inches='tight')

print()
print(result["매수년월"].min(), result["매도년월"].max())
print(pd.DataFrame(recipes))

summary.index.name = "factor"
summary.to_sql("sfw", sqlite3.connect(db_file), if_exists="replace")
summary["strength"] = pow(summary["spearman"] * summary["h10"], 2)
summary = summary.sort_values("strength", ascending=False)
summary

32/32
2002-04 2023-03
   P  GP/P  EQ/P  GP_QoQ  O_QoQ  GP/A_QoQ  O/A_QoQ  R/P  R_QoQ  E_QoQ  \
0 -4     4     2       1      1         1        1  NaN    NaN    NaN   
1 -4     2     2       1      1         1        1  2.0    1.0    1.0   
2 -8     4     4       1      1         1        1  4.0    1.0    1.0   
3 -8     4     4       1      1         1        1  NaN    1.0    1.0   

   R/A_QoQ  E/A_QoQ  
0      NaN      NaN  
1      1.0      1.0  
2      1.0      1.0  
3      1.0      1.0  


,spearman,h3,h5,h10,t3,t5,t10,mdd5,h3-t3,h10-t10,h3-t3+mdd3,h10-t10+mdd10,mean,median,strength
factor,,,,,,,,,,,,,,,
super3,0.955116,0.442213,0.427516,0.402116,-0.124347,-0.106530,-0.089864,-0.541436,0.566560,0.491980,0.038477,-0.005510,0.101603,0.078435,0.147508
super2,0.959148,0.418341,0.421076,0.377613,-0.129516,-0.117658,-0.087658,-0.518173,0.547857,0.465270,-0.003542,-0.062593,0.100879,0.088350,0.131179
super0,0.941590,0.420008,0.407745,0.369740,-0.150465,-0.131997,-0.100159,-0.538897,0.570473,0.469899,0.037739,-0.056902,0.102676,0.092891,0.121204
super1,0.918692,0.427065,0.398100,0.352356,-0.148288,-0.109213,-0.096683,-0.514258,0.575352,0.449039,0.050719,-0.111814,0.101264,0.084526,0.104786
EQ/P,0.911803,0.190926,0.193413,0.195516,-0.150940,-0.129904,-0.080657,-0.552464,0.341866,0.276173,-0.213963,-0.281121,0.082594,0.094694,0.031781
GP/P,0.890177,0.204508,0.202517,0.195880,-0.088115,-0.090015,-0.081750,-0.589156,0.292623,0.277629,-0.304897,-0.287976,0.095200,0.106810,0.030404
O_QoQ,0.828959,0.149313,0.194685,0.187324,0.003346,-0.006478,-0.008210,-0.627823,0.145967,0.195534,-0.512442,-0.400348,0.082022,0.079559,0.024113
R/P,0.866955,0.180246,0.187488,0.175636,-0.111509,-0.100005,-0.063625,-0.568308,0.291755,0.239260,-0.281520,-0.300435,0.084244,0.096647,0.023186
O/P,0.795272,0.191775,0.174409,0.182759,-0.016323,-0.003980,0.006320,-0.600837,0.208097,0.176439,-0.401003,-0.423941,0.098758,0.102533,0.021125


In [195]:
from core.base.quantutil import cagr, mdd
from base.timeutil import YearMonth

_key = ["매수년월", "매도년월"]
benchmark = result.groupby(_key)["수익률"].mean()

factor = "super3"
print(recipes[3])

head = 20
result2 = result.copy()
result2 = result[~result["종목명"].str.endswith("홀딩스")]
result2 = result2[~result2["종목명"].str.endswith("지주")]
# result2 = result2[result2["E/EQ_pct"] > 5]

# result2 = result2[result2["TV/P_pct"] > 5]
# result2 = result2[result2["TV/P_pct"] < 85]

strategy = result2.groupby(_key).apply(
    lambda values: values.sort_values(factor, ascending=False).head(head)["수익률"].mean())
strategy = pd.Series(strategy, index=benchmark.index).fillna(0)

revisions = pd.DataFrame({
    "벤치마크 수익률": benchmark,
    "전략 수익률": strategy,
    "성능": round(strategy - benchmark, 2),
    "벤치마크 누적수익률": (benchmark + 1).cumprod(),
    "전략 누적수익률": (strategy + 1).cumprod()
})


def str_to_ym(_s: str):
    _sp = _s.split("-")
    return YearMonth(int(_sp[0]), int(_sp[1]))


begin = str_to_ym(revisions.index[0][0])
end = str_to_ym(revisions.index[-1][-1])
years = (end.value() - begin.value()) / 12

summary = pd.DataFrame({
    "CAGR": [cagr(1, revisions["전략 누적수익률"][-1], years), cagr(1, revisions["벤치마크 누적수익률"][-1], years)],
    "MDD": [mdd([i2 for _, i2 in revisions.index], revisions["전략 누적수익률"]),
            mdd([i2 for _, i2 in revisions.index], revisions["벤치마크 누적수익률"])],
    "Mean": [revisions["전략 수익률"].mean(), revisions["벤치마크 수익률"].mean()]
}, index=["전략", "벤치마크"])
print(summary)

result3 = result2.groupby(_key).apply(lambda values: values.sort_values(factor, ascending=False).head(head))[
    ["code", "종목명", "수익률", "거래소", "매수일", "매도일", "매수가", "매도가", f"{factor}_pct"]]
result3

{'P': -8, 'GP/P': 4, 'EQ/P': 4, 'R_QoQ': 1, 'GP_QoQ': 1, 'O_QoQ': 1, 'E_QoQ': 1, 'R/A_QoQ': 1, 'GP/A_QoQ': 1, 'O/A_QoQ': 1, 'E/A_QoQ': 1}
          CAGR                                        MDD      Mean
전략    0.412095  (-0.4686698126476754, (2007-09, 2008-10))  0.032064
벤치마크  0.085833  (-0.5734079671946148, (2007-07, 2008-10))  0.009209


code       종목명       수익률   거래소         매수일  \
매수년월    매도년월                                                           
2002-04 2002-05 71      023150     MH에탄올 -0.013793  유가증권  2002-04-30   
                380     006090      사조오양 -0.024421  유가증권  2002-04-30   
                718     025880     케이씨피드  0.038889   코스닥  2002-04-30   
                377     003960      사조대림  0.060615  유가증권  2002-04-30   
                366     011390      부산산업  0.026563  유가증권  2002-04-30   
...                        ...       ...       ...   ...         ...   
2023-02 2023-03 387442  000850      화천기공  0.090116  유가증권  2023-02-28   
                387202  075130      플랜티넷 -0.138699   코스닥  2023-02-28   
                385469  007720    대명소노시즌 -0.037992   코스닥  2023-02-28   
                386281  001540      안국약품 -0.070303   코스닥  2023-02-28   
                386558  037400  우리엔터프라이즈 -0.078640   코스닥  2023-02-28   

                               매도일    매수가      매도가  super3_pct  
매수년월    매도년월                                                    
2002-04 2002-05 71      2002-05-31    725    715.0       100.0  
                380     2002-05-31   2416   2357.0       100.0  
                718     2002-05-31    720    748.0       100.0  
                377     2002-05-31  38621  40962.0       100.0  
                366     2002-05-31   6400   6570.0       100.0  
...                            ...    ...      ...         ...  
2023-02 2023-03 387442  2023-03-31  34400  37500.0        95.0  
                387202  2023-03-31   2920   2515.0        95.0  
                385469  2023-03-31    737    709.0        95.0  
                386281  2023-03-31   8250   7670.0        95.0  
                386558  2023-03-31   1882   1734.0        95.0  

[25100 rows x 9 columns]

In [196]:
result3.loc[result3["수익률"] == -1, "수익률"] = -0.5
total = result3.groupby("code")["수익률"].apply(lambda x: (x + 1).prod()).rename("최종수익률")
count = result3.groupby("code").size().rename("count")
count_pct = np.ceil(count.rank(method="min", pct=True) * 100).rename("count_pct")
count_std = ((count - count.min()) / (count.max() - count.min()) * 20).rename("count_std")
count_std = np.ceil(count_std)

perform_by_count = pd.concat([
    count,
    count_std,
    ((total / 1) ** (1 / count) - 1).rename("평균수익률")
], axis=1)


total = perform_by_count.groupby("count_std")["평균수익률"].apply(lambda x: (x + 1).prod()).rename("총수익률")
count = perform_by_count.groupby("count_std").size().rename("count")

pd.concat([
    total,
    count,
    ((total / 1) ** (1 / count) - 1).rename("평균수익률")
], axis=1)
# x.plot(kind='bar', x='count', y='평균수익률')


,총수익률,count,평균수익률
count_std,,,
0.0,49.795741,81,0.049429
1.0,29665.471790,315,0.033231
2.0,102.725350,179,0.026215
3.0,12.166328,138,0.018271
4.0,7.380364,95,0.021263
5.0,4.714704,66,0.023773
6.0,4.126187,57,0.025178
7.0,2.153554,36,0.021538
8.0,1.678143,36,0.014484


In [ ]:
revisions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

months = [i for _, i in strategy.index]

plt.rc("font", family="Malgun Gothic")
plt.rc('axes', unicode_minus=False)
plt.rcParams['grid.color'] = (0.5, 0.5, 0.5, 0.1)

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2)
fig.set_figwidth(20)
fig.set_figheight(20)

xticks = [1 * i / 10 for i in range(10)] + [1]

ax1.set_title(f"Return/Month")
ax1.set_xlabel("Year/Month")
ax1.set_ylabel("Return")
ax1.set_xticks(np.quantile(np.arange(0, len(revisions)), xticks))
ax1.bar(months, revisions["전략 수익률"])

window = 3
ax2.set_title("Return/Month - Rolling(6)")
ax2.set_xlabel("Year/Month")
ax2.set_ylabel("Return")
ax2.set_xticks(np.quantile(np.arange(0, len(revisions)), xticks))
ax2.bar(months, revisions["전략 수익률"].rolling(window).mean())

ax3.set_title(f"Return/Month")
ax3.set_xlabel("Year/Month")
ax3.set_ylabel("Return(Blue: strategy, Orange: benchmark)")
ax3.set_xticks(np.quantile(np.arange(0, len(revisions)), xticks))
ax3.bar(months, revisions["전략 수익률"])
ax3.bar(months, revisions["벤치마크 수익률"])

ax4.set_title("Return/Month - Rolling(6)")
ax4.set_xlabel("Year/Month")
ax4.set_ylabel("Return(Blue: strategy, Orange: benchmark)")
ax4.set_xticks(np.quantile(np.arange(0, len(revisions)), xticks))
ax4.bar(months, revisions["전략 수익률"].rolling(window).mean())
ax4.bar(months, revisions["벤치마크 수익률"].rolling(window).mean())

ax5.set_title("Perform/Month")
ax5.set_xlabel("Year/Month"), ax5.set_ylabel("Perform")
ax5.set_xticks(np.quantile(np.arange(0, len(revisions)), xticks))
ax5.bar(months, revisions["전략 수익률"] - revisions["벤치마크 수익률"])

ax6.set_title(f"Perform/Month - Rolling({window})")
ax6.set_xlabel("Year/Month")
ax6.set_ylabel(f"Perform - Rolling({window})")
ax6.set_xticks(np.quantile(np.arange(0, len(revisions)), xticks))
ax6.bar(months, (revisions["전략 수익률"] - revisions["벤치마크 수익률"]).rolling(window).mean())

pd.Series({
    "전체 개월 수": len(revisions),
    "초과수익 개월 수": len(revisions[revisions["성능"] > 0]),
    "아웃퍼폼 총합": revisions[revisions["성능"] > 0]["성능"].sum(),
    "언더퍼폼 총합": revisions[revisions["성능"] < 0]["성능"].sum()
})